In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
import pickle

In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='ByM4wBt2n2a30mik3FSbOSs8OqAlYrwk_HV5Q_n8B9Tr',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'carre-donotdelete-pr-xtcjhqrdqvmia9'
object_key = 'autos_preprocessed(clean).csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,Unnamed: 0,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,1,18300.0,coupe,2011.0,manual,190.0,not-declared,125000,5.0,diesel,audi,Yes
1,2,9800.0,suv,2004.0,automatic,163.0,grand,125000,8.0,diesel,jeep,not-declared
2,3,1500.0,small car,2001.0,manual,75.0,golf,150000,6.0,petrol,volkswagen,No
3,4,3600.0,small car,2008.0,manual,69.0,fabia,90000,7.0,diesel,skoda,No
4,5,650.0,limousine,1995.0,manual,102.0,3er,150000,10.0,petrol,bmw,Yes


In [3]:
#print the column names for having a breif idea about the data,
#via shape and noumber of rows and columns
print(df.columns ,df.shape)


Index(['Unnamed: 0', 'price', 'vehicleType', 'yearOfRegistration', 'gearbox',
       'powerPS', 'model', 'kilometer', 'monthOfRegistration', 'fuelType',
       'brand', 'notRepairedDamage'],
      dtype='object') (281934, 12)


In [4]:
#print al the different sellers
#print(df.seller.value_counts())

In [5]:
#remove the seller type having only 3 cars
#df[df.seller != 'gewerblich']

In [6]:
#now all the sellers are same so we can get rid of this column
#df=df.drop('seller',1)

In [7]:
#print al the different sellers
#print(df.offerType.value_counts())

In [8]:
#remove the Offer Type having only 12 listings
#df[df.offerType != 'Gesuch']

In [9]:
#now all the offers are same so we can get rid of this column
#df=df.drop('offerType',1)

In [10]:
#Cars having power less than 50ps and above 900ps seems a little suspicious,
#let's remove them and see what we've got now
print(df.shape)
df = df[(df.powerPS > 50) & (df.powerPS < 900)]

(281934, 12)


In [11]:
print(df.shape)
#around 50000 cars ahave been removed which could have inrouduced error to our data

(281934, 12)


In [12]:
#simlarly, filtering our the cars having registeration years not in the mentioned range
#print(df.shape)
#df = df[(df.yearOfRegistration >= 1950) & (df.yearOfRegistration < 2017)]

In [13]:
#dropping the duplicates from the dataframe and stroing it in a new df.
#here all rows having same value in all the mentioned columns will be deleted and by default,
#only first occurance of anysuch row is kept
new_df = df.copy()
new_df = new_df.drop_duplicates(['price','vehicleType','yearOfRegistration'
                         ,'gearbox','powerPS','model','kilometer','monthOfRegistration','fuelType'
                         ,'notRepairedDamage'])


In [14]:
#after removing duplicates
print(new_df.shape)

(278583, 12)


In [15]:
#As the dataset contained some german words for many features, cahnging them to english
new_df.gearbox.replace(('manuell', 'automatik'), ('manual','automatic'), inplace=True)
new_df.fuelType.replace(('benzin','andere','elektro'),('petrol','others','electric'),inplace=True)
new_df.vehicleType.replace(('kleinwagen', 'cabrio','kombi','andere'),
                           ('small car','convertible','combination','others'),inplace=True)
new_df.notRepairedDamage.replace(('ja','nein'),('Yes','No'),inplace=True)

In [16]:
#### Removing the outliers
new_df = new_df[(new_df.price >= 100) & (new_df.price <= 150000)]

In [17]:
#Filling NaN values for columns whose data might not be there with the information provider,
#which might lead to some variance but our model
#but we will still be able to give some estimate to the user
new_df['notRepairedDamage'].fillna(value='not-declared', inplace=True)
new_df['fuelType'].fillna(value='not-declared', inplace=True)
new_df['gearbox'].fillna(value='not-declared', inplace=True)
new_df['vehicleType'].fillna(value='not-declared', inplace=True)
new_df['model'].fillna(value='not-declared', inplace=True)

In [18]:
#can save the csv for future purpose. 
new_df.to_csv("autos_preprocessed.csv")

In [19]:
#Columns which contain categorical values, which we'll need to convert via label encoding
labels = ['gearbox', 'notRepairedDamage', 'model', 'brand', 'fuelType', 'vehicleType']

In [20]:
#looping over the labels to do the label encoding for all at once and 
#saving the LABEL ENCODING FILES
mapper = {}
for i in labels:
    mapper[i] = LabelEncoder()
    mapper[i].fit(new_df[i])
    tr = mapper[i].transform(new_df[i])
    np.save(str('classes'+i+'.npy'), mapper[i].classes_)
    print(i,":",mapper[i])
    new_df.loc[:, i + '_labels'] = pd.Series(tr, index=new_df.index)

gearbox : LabelEncoder()
notRepairedDamage : LabelEncoder()
model : LabelEncoder()
brand : LabelEncoder()
fuelType : LabelEncoder()
vehicleType : LabelEncoder()


In [21]:
#Final data to be put in a new dataframe called "LABELED",
labeled = new_df[ ['price'
                        ,'yearOfRegistration'
                        ,'powerPS'
                        ,'kilometer'
                        ,'monthOfRegistration'
                        ] 
                    + [x+"_labels" for x in labels]]

In [22]:
print(labeled.columns)

Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


# Train-Test Split

In [23]:
#Storing price in Y and rest of the data in X
Y = labeled.iloc[:,0].values
X = labeled.iloc[:,1:].values

In [24]:
#need to reshape the Y values
Y = Y.reshape(-1,1)

In [25]:
from sklearn.model_selection import cross_val_score, train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 3)

# Model building and Fitting

In [26]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
regressor = RandomForestRegressor(n_estimators=1000,max_depth=10,random_state=34)

In [27]:
regressor.fit(X_train, np.ravel(Y_train,order='C'))

RandomForestRegressor(max_depth=10, n_estimators=1000, random_state=34)

In [28]:
#predicting the values fo test test
y_pred = regressor.predict(X_test)

In [29]:
#printing the Accuraccy for test set
print(r2_score(Y_test,y_pred))

0.8387903149180778


In [30]:
#for testing on user input values
y_pred1 = regressor.predict([[2011,190,125000,5,1,0,163,1,3,3]])
#predticting price for a user input values
print(y_pred1)

[19070.72434796]


In [31]:
#!pip install ibm_watson_machine_learning

In [32]:
from ibm_watson_machine_learning import APIClient
wml_credential={
    "url" : "https://us-south.ml.cloud.ibm.com" ,
    "apikey": "Z0tkHOU_LhTROPCSHN-GlbTrQYB-Vqk-bOml6rpBtf16"
}
client=APIClient(wml_credential)

In [33]:
def space_Name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [34]:
space_uid=space_Name(client,'models')


In [35]:
print("space_uid"+space_uid)

space_uidf0505d67-e053-4df2-96e0-fc5b8bec76a8


In [36]:
client.set.default_space(space_uid)

'SUCCESS'

In [37]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [38]:
software_spec_uid=client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [39]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()

In [40]:
model_details = client.repository.store_model(model=model,meta_props={
client.repository.ModelMetaNames.NAME: 'Resalemodel',
client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0',
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid}
                                             )
                                                

In [41]:
model_id = client.repository.get_model_id(model_details)

In [42]:
model_id

'ca31ff7b-1fef-4ff9-b414-deba77f860c9'

In [45]:
client.repository.download('ca31ff7b-1fef-4ff9-b414-deba77f860c9','Deployed_Model.tar.gz')

Successfully saved model content to file: 'Deployed_Model.tar.gz'


'/home/wsuser/work/Deployed_Model.tar.gz'